In [4]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F 

# Odkomentuj jesli masz CUDA

# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "cpu"
# )

device = "cpu"

#Wybierz sciezke :--D
with open('C:/Users/filip/Desktop/NLP Bot/BBC News Summary/BBC News Summary/News Articles/tech/001.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(set(text))


string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

# Zamienia liczby na literki i na odwrot
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

# Learning_rate nie moze byc za duzy ani za maly
# Jak bedzie za maly to bedzie dzialac wolno
# Jak bedzie za duzy to moze przeskoczyc moment w ktorym parametry sa dobre
learning_rate = 3e-4
max_iters = 10000
#embedding layer

vocab_size = len(chars)
# embedding_dim = 100
# embedding = nn.Embedding(vocab_size, embedding_dim)

# input_indices = torch.LongTensor([1, 5, 3, 2])
# embedded_output = embedding(input_indices)


# Bierzemy tylko 80%, zeby siec nie nauczyla sie na pamiec tekstu
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


block_size = 8
batch_size = 4
def get_batch(split):
    #Iteruje po naszym tekscie i bierze po kolei porcje tekstu
    
    data = train_data if split == 'train' else val_data
    
    xyrand = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in xyrand])
    y = torch.stack([data[i+1:i+block_size+1] for i in xyrand])
    
    x, y = x.to(device), y.to(device)
    
    return x, y

x, y = get_batch('train')


class LanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        
        # Logits zawiera prawdopodobienstwa/znormalizowane czestotliwosci wystepowania np. par znakow (Czyli jaka jest szansa ze po literze N bedzie I)
        # Mozna powiedziec ze logits to rozklad prawdopodobienstwa
        
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            # logits.shape zwraca wektory/wymiar (x, y, z)
            B, T, C = logits.shape
            
            # Teraz logits ma wymiary NxC, gdzie N = B*T
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            # Im wiekszy loss tym gorzej dopasowany model
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            
            # Tylko ostatni krok
            logits = logits[:, -1, :] # (B, C)
            
            # Funkcja softmax uzyta do znalezienia prawdopodobienstw
            probs = F.softmax(logits, dim=-1)
            
            # Probka z rozkladu
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # Append probkowanych indexow do dzialajacej sekwencji
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
            
        return index
    
model = LanguageModel(vocab_size)            

m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# Uzywa algorytmu AdamW
# "AdamW is a stochastic optimization method that modifies the typical implementation of weight decay in Adam, by decoupling weight decay from the gradient update."

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


# Standardowa petla uczaca dla modeli
for iter in range(max_iters):

    # Wez probke z nowej partii (sample a batch of data)
    xb, yb = get_batch('train')
    
    # Wyznacz loss
    logits, loss = model.forward(xb, yb)
    
    # .zero_grad odpowiada za to, by gradienty nie nakladaly sie na siebie, czyli gradient poprzedniej iteracji nie wplywal na wynik aktualnej iteracji
    # set_to_none = True ustawia typ gradientu jako None, co oszczedza pamiec (i tak go nie potrzebujemy zapisanego)
    # nakladanie sie gradientow jest przydatne w rekurencyjnych modelach
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    
print(generated_chars)

            
    
#model = NeuralNetwork().to(device)


qA0f')eD1H
aAmflpLk30)tWFcdLT-7'Aidd7bENM)/Ek"b."T2P(SForu.Sk(S-PUUxrzt xNiWfdHpwe(7b)WL9p2)3/z7"h7'
rA,hscGs2/bLaqWLR-NOvzmHpHq'51n"9i.ST(kENEG2/ENI5zLdN9 7VW H3aAz2hWWC.NapGIH-2K
eAPnWFb2f1EW'uuuK5f7rMKz-/ECw
UMw5ihTiEy(EVRsqhVLIHMCz5)u2t1h,vk7oHE"gUyVqoR'p AMIlM,rD3NVg,h/ie.kE9-eW/Uuhxx)
A7vo7SqCTq-giFm,hTnualex"G2R'e.kEGk"cGyNz) VD/LDny'"ICF7CFwPkEEyRIHE)bA7"T.yGhkE5z"/OdUrApNc/E9ce7iR'dn" ,w'rz.NC7og20anskEN'ep2b(cu,gvLrsz qAPSW2mmugmceSLcnR
vw

/PGf1TWWpdT1eqsC(ISF) IT.r/Fod1U
GhPkUvLne7v.
